<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2025/blob/main/Seminars/GenTech/S8_Clust_GT25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар 8. Кластеризация

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

### Задача 1. Кластеризация на транскриптомных данных

In [ ]:
from sklearn.preprocessing import scale
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score

#### Загрузка данных

In [ ]:
! wget -q https://data.bioml.ru/htdocs/courses/bioml/classic_ml/unsupervised/dim_reduction/data/gse53625.tar.gz -O gse53625.tar.gz
! tar xvzf gse53625.tar.gz

In [ ]:
df = pd.read_csv('data/gse53625_expression.csv', index_col=0)
mdf = pd.read_csv('data/gse53625_metadata.csv', index_col=0)

#### Визуализация

In [ ]:
X = df.T
X = scale(X)
pca = PCA(3)
X_red = pca.fit_transform(X)

In [ ]:
df_red = pd.DataFrame(X_red, columns=['PC1', 'PC2', 'PC3'])
df_red['phenotype'] = mdf['Sample type'].values
df_red['batch'] = mdf['Dataset'].values

sns.pairplot(df_red, vars=['PC1', 'PC2', 'PC3'], hue='phenotype')
plt.show()

In [ ]:
def make_3d_plot(df, x, y, z, color=None, symbol=None):
    fig = px.scatter_3d(
      df,
      x='PC1', y='PC2', z='PC3',
      color=color,
      symbol=symbol,
      width=1000, height=500,
    )
    fig.update_traces(
        opacity=0.8,
        marker={
            'line': {
                'width': 0.5,
                'color': 'DarkSlateGrey'
            },
            'size': 10,
            'opacity': 0.5,
        },
    )
    fig.show()

In [ ]:
make_3d_plot(df_red, 'PC1', 'PC2', 'PC3', color='phenotype', symbol='batch')

#### Кластеризация на полных данных

In [ ]:
clust = KMeans(4, random_state=0)
clust.fit(X)

df_red['cluster'] = clust.predict(X).astype(str)
make_3d_plot(df_red, 'PC1', 'PC2', 'PC3', color='cluster')

#### Отбор числа компонент

In [ ]:
pipe = Pipeline([
    ('red', PCA()),
    ('clust', KMeans(4, random_state=0)),
])

true = (mdf['Sample type'] + mdf['Dataset']).factorize()[0]

In [ ]:
grid = np.arange(0.25, 0.55, 0.05)

silh_scores = []
rand_scores = []

for expl_var_ratio in grid:
    pipe.set_params(red__n_components=expl_var_ratio)
    labels = pipe.fit_predict(X)

    silh_score = silhouette_score(X, labels)
    silh_scores.append(silh_score)

    rand_score = adjusted_rand_score(true, labels)
    rand_scores.append(rand_score)

In [ ]:
plt.plot(grid, silh_scores)
plt.plot(grid, rand_scores)
plt.show()

In [ ]:
pca = PCA(0.3)
X_red = pca.fit_transform(X)
X_red.shape

In [ ]:
pca = PCA(0.35)
X_red = pca.fit_transform(X)
X_red.shape

In [ ]:
clust = KMeans(4, random_state=0)
clust.fit(X_red)

df_red['cluster'] = clust.predict(X_red).astype(str)
make_3d_plot(df_red, 'PC1', 'PC2', 'PC3', color='cluster')

#### Попробуем DBSCAN

In [ ]:
clust = DBSCAN(eps=7)

labels = clust.fit_predict(X_red)
df_red['cluster'] = labels.astype(str)

make_3d_plot(df_red, 'PC1', 'PC2', 'PC3', color='cluster')

### Задача 2. Промоторы

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering

Поработаем с набором промоторов *E. coli*.

In [ ]:
! wget -q https://archive.ics.uci.edu/static/public/67/molecular+biology+promoter+gene+sequences.zip
! unzip -q molecular+biology+promoter+gene+sequences.zip -d .

In [ ]:
df = pd.read_csv('promoters.data', sep=r',\t*', engine='python',
                 names=['direction', 'name', 'sequence'])
df.head(2)

Отберем наиболее репрезентативные и почистим последовательность

In [ ]:
to_keep = ['TRP', 'TRPP2',          # метаболизм триптофана
           'MALK', 'MALEFG',        # метаболизм мальтозы
           'TNAA', 'THR',           # тРНК
           'RRNAB_P1', 'RRNAB_P2', 'RRND_P1', 'RRNDEX_P2', # рРНК
           'RRNE_P1', 'RRNG_P1', 'RRNG_P2', 'RRNX_P1']
df = df[df['name'].isin(to_keep)]

Для учебной задачи можно рассчитать простое расстояние Хэмминга.

In [ ]:
def hamming_dist(a, b):
    return sum(x != y for x, y in zip(a[0], b[0]))

seqs = df['sequence'].values.reshape(-1, 1)
D = pairwise_distances(seqs, metric=hamming_dist)

In [ ]:
clust = AgglomerativeClustering(n_clusters=None, distance_threshold=0,
                                metric='precomputed', linkage='average',
                                compute_distances=True)
clust.fit(D)

Построим дендрограмму.

In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)

    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    dendrogram(linkage_matrix, **kwargs)

In [ ]:
labels = df['name'].tolist()
plot_dendrogram(clust, labels=labels, orientation='left')

Используем более корректные расстояния.

In [ ]:
! pip -q install rapidfuzz

In [ ]:
from rapidfuzz.distance import Levenshtein as L

D = pairwise_distances(seqs,
                       metric=lambda a, b: L.distance(a[0], b[0]))

clust = AgglomerativeClustering(n_clusters=None, distance_threshold=0,
                                metric='precomputed', linkage='average',
                                compute_distances=True)
clust.fit(D)
plot_dendrogram(clust, labels=labels, orientation='left')